## Imports

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
import torch.nn as nn
import numpy as np
import math
from dataclasses import dataclass

In [36]:
hf_gpt2 = AutoModelForCausalLM.from_pretrained("gpt2")
hf_tokenizer = AutoTokenizer.from_pretrained("gpt2")
hf_config = AutoConfig.from_pretrained("gpt2")

print(hf_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 50257
}



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## GPT-2 inputs & outputs at a glance

In [ ]:
sentence = "I love this gelateria."

hf_tokenizer.tokenize(sentence)

['I', 'Ġlove', 'Ġthis', 'Ġgel', 'ater', 'ia', '.']

In [ ]:
hf_tokenizer(sentence)

{'input_ids': [40, 1842, 428, 20383, 729, 544, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
vocab = hf_tokenizer.vocab
vocab

{'ĠEthereum': 20313,
 'Ġannoun': 2335,
 'Ġcomments': 3651,
 'ĠFalse': 10352,
 'err': 8056,
 'tha': 12898,
 'Ġcorpses': 26750,
 'ĠMini': 12558,
 'ĠDaniels': 28162,
 'Ġcarve': 36717,
 'Â¢': 44359,
 'Ġmere': 5019,
 'Ġdefeated': 9772,
 'Ġmotive': 20289,
 'Ġfrivol': 43418,
 'Ġrenew': 6931,
 'Ġstationary': 31607,
 'ĠSchwarzenegger': 47507,
 'ĠBeans': 49740,
 'ĠLub': 40753,
 'ĠCross': 6372,
 'ãģ®ç': 17683,
 'ĠDed': 35023,
 'ĠCot': 42266,
 'ãĢģ': 23513,
 'ĠAugustine': 36158,
 'RH': 48587,
 'Ġcand': 2658,
 'ĠPurdue': 34699,
 'As': 1722,
 'Ġneglect': 17985,
 'ĠSenegal': 49657,
 'ĠMod': 3401,
 'birth': 24280,
 'Ġchallenges': 6459,
 'Ġmech': 36571,
 'ĠWARNING': 39410,
 'ĠDevices': 29362,
 'ĠNed': 35754,
 'ftime': 31387,
 'DO': 18227,
 'Ġhalted': 27771,
 'Ġboredom': 43515,
 'Ġinterpreter': 28846,
 'ĠBunker': 44761,
 'Ġaborted': 46847,
 'essions': 6202,
 'ĠOwner': 23853,
 'Ġlightsaber': 45282,
 'Ġhashes': 46621,
 'Ġnovel': 5337,
 'Ġdetail': 3703,
 'Ġfifteen': 17280,
 'ĠJazeera': 32409,
 'ittees': 13

In [ ]:
inputs  = hf_tokenizer.encode(sentence, return_tensors="pt")

outputs = hf_gpt2(inputs)

print(outputs.keys())

print(outputs.logits.shape)


odict_keys(['logits', 'past_key_values'])
torch.Size([1, 7, 50257])


In [ ]:
outputs

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ -39.3084,  -39.0100,  -41.8374,  ...,  -46.9337,  -44.9074,
           -39.5149],
         [ -84.4961,  -85.0687,  -90.6138,  ...,  -91.6659,  -93.1035,
           -87.5303],
         [ -89.1614,  -91.6797,  -97.9510,  ..., -101.9002, -100.6931,
           -94.4206],
         ...,
         [ -72.7832,  -74.8255,  -81.6131,  ...,  -87.7673,  -83.6719,
           -75.3573],
         [ -86.5229,  -91.0898,  -97.6117,  ..., -102.8940, -100.2883,
           -92.0316],
         [-104.3630, -106.1659, -105.8746,  ..., -116.4454, -116.2705,
          -100.3265]]], grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-1.5577,  2.0585,  1.3060,  ..., -1.3825, -0.6334,  1.2624],
          [-1.4860,  2.7392,  1.4598,  ..., -2.6422, -2.0976,  1.9058],
          [-2.0344,  2.3477,  1.8740,  ..., -1.2935, -1.8629,  2.1281],
          ...,
          [-1.5971,  2.4094,  1.6590,  ..., -1.6091, -2.1870,  2.3390],
          [-3.0018,  

## Transformer Architecture

![](https://raw.githubusercontent.com/DuyguA/GPT-2-for-GDE-day/refs/heads/main/trans.png)

![](https://raw.githubusercontent.com/DuyguA/GPT-2-for-GDE-day/refs/heads/main/gptdetail.png)

## Code

### Config

We define a stripped down config for our model

In [8]:

@dataclass
class Config:
    d_model: int = 768
    layer_norm_eps: float = 1e-5
    d_vocab: int = 50257
    init_range: float = 0.02
    n_ctx: int = 1024
    d_head: int = 64
    d_mlp: int = 3072
    n_heads: int = 12
    n_layers: int = 12

cfg = Config()
print(cfg)

Config(d_model=768, layer_norm_eps=1e-05, d_vocab=50257, init_range=0.02, n_ctx=1024, d_head=64, d_mlp=3072, n_heads=12, n_layers=12)


['I', 'Ġlove', 'Ġthis', 'Ġgel', 'ater', 'ia', '.']

[40, 1842, 428, 20383, 729, 544, 13]

Input: batch x token_ids : (batch_size, seq_len)  # (1, 7)

After embeddings and all layers: (batch_size, seq_len, d_model) # (1, 7, 768)

## Embedding

In [20]:
class Embed(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.embedding_table = nn.Embedding(cfg.d_vocab, cfg.d_model) # 50257, 768

    def forward(self, tokens):
        # tokens: [batch, position]
        embeddings = self.embedding_table(tokens)
        return embeddings

tokens = torch.tensor([[40, 1842, 428, 20383, 729, 544, 13]])
print(tokens.shape)  # (batch_size, seq_len)
embedding_layer = Embed(cfg)
embeddings = embedding_layer(tokens)
print(embeddings.shape)  #(batch_size, seq_len, d_model)
embeddings


torch.Size([1, 7])
torch.Size([1, 7, 768])


tensor([[[ 1.4091, -0.4358, -1.1750,  ..., -1.4225, -1.9780,  1.0570],
         [-0.4358, -0.9107, -0.2517,  ..., -1.3571,  0.8217,  1.2522],
         [-1.2425,  0.3257, -0.4472,  ...,  0.2619, -1.4548,  0.1175],
         ...,
         [ 0.5002, -0.0336, -1.1452,  ...,  0.6873,  1.1604,  0.5493],
         [ 0.0665, -0.9969, -1.5832,  ...,  0.1536,  0.2693,  0.8137],
         [-1.0206, -0.9477, -0.5099,  ..., -0.4143, -1.0984, -0.3275]]],
       grad_fn=<EmbeddingBackward0>)

## Positional Embedding

In [21]:
class PosEmbed(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.pos_embeddings = nn.Embedding(cfg.n_ctx, cfg.d_model)  # 1024, 768


    def forward(self, tokens):
        batch_size, seq_length = tokens.size()  # tokens shape: (batch_size, seq_length)
        pos_embed = self.pos_embeddings(torch.arange(seq_length))  # (seq_length, d_model) 0,1...7
        pos_embed = torch.einsum("sd,b->bsd", pos_embed, torch.ones(batch_size)) # (batch_size, seq_len, d_model)
        return pos_embed

tokens = torch.tensor([[40, 1842, 428, 20383, 729, 544, 13]])

pos_embed = PosEmbed(cfg)
pos_embed(tokens).shape

torch.Size([1, 7, 768])

## LayerNorm

BatchNorm normalizes each feature within a batch of samples, while LayerNorm normalizes all features within each sample.



![](https://raw.githubusercontent.com/DuyguA/GPT-2-for-GDE-day/refs/heads/main/batch.png)

In [22]:
class LayerNorm(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.layer_norm = nn.LayerNorm(cfg.d_model, cfg.layer_norm_eps)

    def forward(self, X):
        X = self.layer_norm(X)
        return X

inputs = torch.randn(1, 7, 768)
layer_norm = LayerNorm(cfg)
normalized = layer_norm(inputs)
print(normalized.shape)

torch.Size([1, 7, 768])


## Attention

![](https://raw.githubusercontent.com/DuyguA/GPT-2-for-GDE-day/refs/heads/main/ir_system.png)



only one key             |  all keys
:-------------------------:|:-------------------------:
![](https://raw.githubusercontent.com/DuyguA/GPT-2-for-GDE-day/refs/heads/main/simple_attention.jpg)  |  ![](https://raw.githubusercontent.com/DuyguA/GPT-2-for-GDE-day/refs/heads/main/matrix.jpg)

In [6]:
class SimpleAttention(nn.Module):
  def __init__(self):
    super().__init__()


  def forward(self, q, sentence):
    # q: single token vector, shape (1, d)
    # s: sentence token, shape (n, d)
    scores = torch.tensor([torch.dot(q, s_i) for s_i in sentence])
    print(scores, "scores")
    attention_weights = torch.softmax(scores, dim=-1)
    print(attention_weights, "attention weights")
    context_vector = sum(attention_weight * s_i for attention_weight, s_i in zip(attention_weights, sentence))
    return context_vector

q = torch.tensor([-1, 2, 1], dtype=torch.float32)
sentence = torch.tensor([[-2, 2, -4], [8, 0, 0], [-1, 2, 1], [0, 4, 1]], dtype=torch.float32)
simp_attn = SimpleAttention()
cv = simp_attn(q, sentence)
print(cv)

tensor([ 2., -8.,  6.,  9.]) scores
tensor([8.6788e-04, 3.9402e-08, 4.7385e-02, 9.5175e-01]) attention weights
tensor([-0.0491,  3.9035,  0.9957])



self attention formula             |  K, Q, V again
:-------------------------:|:-------------------------:
![](https://raw.githubusercontent.com/DuyguA/GPT-2-for-GDE-day/refs/heads/main/qkv.png)  |  ![](https://raw.githubusercontent.com/DuyguA/GPT-2-for-GDE-day/refs/heads/main/qkv2.png)

In [11]:
class SelfAttention(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.cfg = cfg
    self.W_Q = nn.Linear(cfg.d_model, cfg.d_model)
    self.W_K = nn.Linear(cfg.d_model, cfg.d_model)
    self.W_V = nn.Linear(cfg.d_model, cfg.d_model)

  def forward(self, X):
    Q = self.W_Q(X) # query (batch_size, m, d_model)  m=n=7
    K = self.W_K(X)  # key (batch_size, n, d_model)
    V = self.W_V(X)  # value (batch_size, n, d_model)

    scores = torch.einsum("bmd,bnd->bmn", Q, K)
    scores = scores / math.sqrt(self.cfg.d_model)
    print(scores.shape, "scores shape")
    print(scores)
    attention_weights = torch.softmax(scores, dim=-1)
    context_vector = torch.einsum("bmn,bnd->bmd", attention_weights, V)
    return context_vector


X = torch.randn(1, 7, 768)
self_attn = SelfAttention(cfg)
context_vector = self_attn(X)
print(context_vector.shape, "context vector shape")


torch.Size([1, 7, 7]) scores shape
tensor([[[ 0.2775,  0.3087, -0.5478, -0.2229,  0.4653, -0.2421, -0.1320],
         [ 0.0221,  0.1104, -0.3745, -0.0604, -0.4670,  0.8271, -0.0383],
         [ 0.7995,  0.3250,  0.6393,  0.7838, -0.5243, -0.1314, -0.1234],
         [ 0.0690, -0.4545,  0.0610,  0.0420,  0.1379,  0.6444,  0.2420],
         [-0.4419,  0.2479, -0.2990, -0.2699, -0.3189, -0.0769,  0.3399],
         [ 0.4183,  0.0572, -0.1432,  0.0410, -0.4053,  0.1566, -0.0789],
         [ 0.4848, -0.0185, -0.1224,  0.1823,  0.4818,  0.1109, -0.0144]]],
       grad_fn=<DivBackward0>)
torch.Size([1, 7, 768]) context vector shape


In [28]:
class CausalAttention(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.cfg = cfg
    self.W_Q = nn.Linear(cfg.d_model, cfg.d_model)
    self.W_K = nn.Linear(cfg.d_model, cfg.d_model)
    self.W_V = nn.Linear(cfg.d_model, cfg.d_model)

  def forward(self, X):
    Q = self.W_Q(X) # query (batch_size, m, d_model)  m=n=7
    K = self.W_K(X)  # key (batch_size, n, d_model)
    V = self.W_V(X)  # value (batch_size, n, d_model)

    scores = torch.einsum("bmd,bnd->bmn", Q, K)
    scores = scores / math.sqrt(self.cfg.d_model)

    causal_mask = torch.triu(torch.ones_like(scores), diagonal=1).bool()
    #print(causal_mask.shape, "causal mask shape")
    #print(causal_mask, "causal mask")
    scores = scores.masked_fill(causal_mask == 1, float("-inf"))
    #print(scores, "scores")

    attention_weights = torch.softmax(scores, dim=-1)
    context_vector = torch.einsum("bmn,bnd->bmd", attention_weights, V)
    return context_vector


X = torch.randn(1, 7, 768)
self_attn = CausalAttention(cfg)
context_vector = self_attn(X)


## MLP

![](https://raw.githubusercontent.com/DuyguA/GPT-2-for-GDE-day/refs/heads/main/gelu.png)

In [23]:
class MLP(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.linear1 = nn.Linear(cfg.d_model, cfg.d_mlp)  # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_mlp)
        self.linear2 = nn.Linear(cfg.d_mlp, cfg.d_model)  # (batch_size, seq_len, d_mlp) -> (batch_size, seq_len, d_model)
        self.gelu = nn.GELU()

    def forward(self, X):
      X = self.linear1(X)
      X = self.gelu(X)
      X = self.linear2(X)
      return X

inputs = torch.randn(1, 7, 768)
mlp = MLP(cfg)
mlp_out = mlp(inputs)
print(mlp_out.shape)

torch.Size([1, 7, 768])


## Transformer Block

In [29]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg

        self.ln1 = LayerNorm(cfg)
        self.attn = CausalAttention(cfg)
        self.ln2 = LayerNorm(cfg)
        self.mlp = MLP(cfg)

    def forward(self, X):
        # input size [batch, seq_len, d_model]
        resid_pre = self.ln1(X)
        attn_out = self.attn(resid_pre)
        resid_mid = X + attn_out

        normalized_resid_mid = self.ln2(resid_mid)
        mlp_out = self.mlp(normalized_resid_mid)
        resid_post = resid_mid + mlp_out
        return resid_post

inputs = torch.randn(1, 7, 768)
trans_block = TransformerBlock(cfg)
final_out = trans_block(inputs)
print(final_out.shape)

torch.Size([1, 7, 768])


## Unembedding

In [30]:
class Unembedding(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.unembed = nn.Linear(cfg.d_model, cfg.d_vocab) # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_vocab) (1, 7, 768) -> (1, 7, 50257)


    def forward(self, X):
        # X [batch, position, d_model)
        logits = self.unembed(X)
        return logits

inputs = torch.randn(1, 7, 768)
unembed = Unembedding(cfg)
logits = unembed(inputs)
print(logits.shape)

print(logits[0, 1, :])

torch.Size([1, 7, 50257])
tensor([ 0.1201,  0.5953, -0.0510,  ..., -0.0284,  0.0367,  0.9337],
       grad_fn=<SliceBackward0>)


## All together

In [34]:
class OurGPT(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.embed = Embed(cfg)
        self.pos_embed = PosEmbed(cfg)
        self.blocks = nn.ModuleList([TransformerBlock(cfg) for _ in range(cfg.n_layers)])
        self.ln_final = LayerNorm(cfg)
        self.unembedding = Unembedding(cfg)

    def forward(self, tokens):
        # tokens [batch, seq_len]
        embed = self.embed(tokens)
        pos_embed = self.pos_embed(tokens)
        residual = embed + pos_embed
        for block in self.blocks:
            residual = block(residual)
        normalized_resid_final = self.ln_final(residual)
        logits = self.unembedding(normalized_resid_final)
        # logits have shape [batch, seq_len, vocab_size]
        return logits


tokens = torch.tensor([[40, 1842, 428, 20383, 729, 544, 13]])
ourgpt = OurGPT(cfg)
logits = ourgpt(tokens)
print(logits.shape)

print(logits[0, 2, :])

torch.Size([1, 7, 50257])
tensor([ 0.1842, -0.3905, -0.3457,  ...,  0.2911,  0.2378, -1.0478],
       grad_fn=<SliceBackward0>)


## A bit of loss calculation and text generation

Take a test string - the intro paragraph of today's featured Wikipedia article. Let's calculate the loss!

In [37]:
test_text = "Prague is the capital and largest city of the Czech Republic and the historical capital of Bohemia. Situated on the Vltava river, Prague is home to about 1.4 million people. The city has a temperate oceanic climate, with relatively warm summers and chilly winters. "

In [42]:
test_tokens = hf_tokenizer.encode(test_text, return_tensors="pt")
print(test_tokens)
test_logits = ourgpt(test_tokens)

tensor([[ 6836,  2064,   318,   262,  3139,   290,  4387,  1748,   286,   262,
         16639,  2066,   290,   262,  6754,  3139,   286, 45560,   544,    13,
         21131,  6605,   319,   262,   569,  2528,  4170,  7850,    11, 37079,
           318,  1363,   284,   546,   352,    13,    19,  1510,   661,    13,
           383,  1748,   468,   257,  4124,   378,  9151,   291,  4258,    11,
           351,  5365,  5814, 43285,   290, 49018, 45764,    13,   220]])


In [44]:
def cross_entropy_loss(logits, tokens):
    # Measure next token loss
    # Logits have shape [batch, position, d_vocab]
    # Tokens have shape [batch, position]
    log_probs = logits.log_softmax(dim=-1)
    pred_log_probs = log_probs[:, :-1].gather(dim=-1, index=tokens[:, 1:].unsqueeze(-1)).squeeze(-1)
    return -pred_log_probs.mean()

loss = cross_entropy_loss(test_logits, test_tokens)
print(loss)


tensor(11.0288, grad_fn=<NegBackward0>)


In [45]:
loss = nn.CrossEntropyLoss()(test_logits[:, :-1].transpose(1, 2), test_tokens[:, 1:])
print(loss)

tensor(11.0288, grad_fn=<NllLoss2DBackward0>)


In [47]:

for i in range(20):
    test_tokens = test_tokens = hf_tokenizer.encode(test_text, return_tensors="pt")
    test_logits = ourgpt(test_tokens)
    next_token_id = test_logits[:, -1, :].argmax(dim=-1).unsqueeze(0)
    next_token = hf_tokenizer.decode(next_token_id.tolist()[0])
    print(next_token, "next token")
    test_text += next_token
    print(test_text, "new text")
    print("===============")

 disqual next token
Prague is the capital and largest city of the Czech Republic and the historical capital of Bohemia. Situated on the Vltava river, Prague is home to about 1.4 million people. The city has a temperate oceanic climate, with relatively warm summers and chilly winters.   rpm oine  Stack  quint  epic  Russell  explained Rule  dependency ynam  Eng  circ  bury  democr  inadequ  Har inv  Elm react oine disqual new text
 vigilance next token
Prague is the capital and largest city of the Czech Republic and the historical capital of Bohemia. Situated on the Vltava river, Prague is home to about 1.4 million people. The city has a temperate oceanic climate, with relatively warm summers and chilly winters.   rpm oine  Stack  quint  epic  Russell  explained Rule  dependency ynam  Eng  circ  bury  democr  inadequ  Har inv  Elm react oine disqual vigilance new text
 tally next token
Prague is the capital and largest city of the Czech Republic and the historical capital of Bohemia. Si